In [1]:
!pip uninstall keras
!pip install keras==2.3.1

Found existing installation: keras 2.7.0
Uninstalling keras-2.7.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? y
  Successfully uninstalled keras-2.7.0
     |████████████████████████████████| 377 kB 5.2 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires keras<2.8,>=2.7.0rc0, but you have keras 2.3.1 which is incompatible.


In [2]:
!pip uninstall tensorflow
!pip install tensorflow==2.1.0

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? 2
Your response ('2') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 421.8 MB 22 kB/s 
     |████████████████████████████████| 448 kB 46.2 MB/s 
     |████████████████████████████████| 3.8 MB 45.1 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=8f5f7fb420758915c4d8add196b40b15ce6a31cf756d3bb1e732e9c8a880bd58
  Stored in directory: /root/.cache/p

In [3]:
!pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 15.7 MB 70.8 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
tensorflow-probability 0.15.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which 

In [4]:
!pip install matchzoo

     |████████████████████████████████| 91 kB 4.2 MB/s 
  Created wheel for matchzoo: filename=MatchZoo-2.2.0-py3-none-any.whl size=151779 sha256=7a31de3e794a9c648ae9f46af2cc3e20ff499d449a8073505ef19b1a213e6154
  Stored in directory: /root/.cache/pip/wheels/46/35/e3/70d4fccf0cfa8a617a50d37b3da011fe34bd977f7e126e9878
Successfully built matchzoo


In [5]:
### INIT ###

import keras
import pandas as pd
import numpy as np
import matchzoo as mz
import json

print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task='ranking')
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task='ranking', filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task='ranking', filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision(),
    mz.metrics.AveragePrecision(),
    mz.metrics.MeanReciprocalRank(),
    mz.metrics.Precision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

print("loading embedding ...")
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
print("embedding loaded as `glove_embedding`")

Using TensorFlow backend.


data loading ...
7094272/7094233 [==============================] - 0s 0us/step
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0), average_precision(0.0), mean_reciprocal_rank(0.0), precision@1(0.0)]
loading embedding ...
862183424/862182613 [==============================] - 160s 0us/step
embedding loaded as `glove_embedding`


In [6]:
def append_params_to_readme(model):
    import tabulate
    
    with open('README.rst', 'a+') as f:
        subtitle = model.params['model_class'].__name__
        line = '#' * len(subtitle)
        subtitle = subtitle + '\n' + line + '\n\n'
        f.write(subtitle)
        
        df = model.params.to_frame()[['Name', 'Value']]
        table = tabulate.tabulate(df, tablefmt='rst', headers='keys') + '\n\n'
        f.write(table)

  ### END INIT ### 

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
### KRNM ###

preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9440.18it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5126.06it/s]
Building Vocabulary from a datapack.: 100%|██████████| 473697/473697 [00:00<00:00, 3112396.17it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9497.44it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5279.89it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 53076.94it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9782.17it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [9]:
preprocessor.context

{'embedding_input_dim': 17483,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fd3ee9f8350>,
 'input_shapes': [(10,), (40,)],
 'vocab_size': 17483,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fd3efbcf390>}

In [10]:
model = mz.models.KNRM()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()

In [11]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [12]:
pred_x, pred_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x), model_save_path='./krnm_pretrained_model/')

In [13]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 255


In [14]:
%%time
history = model.fit_generator(train_generator, epochs=15, callbacks=[evaluate], workers=30, use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
255/255 [==============================] - 12s 45ms/step - loss: 1.3485
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5300980680404833 - normalized_discounted_cumulative_gain@5(0.0): 0.5935810749186005 - mean_average_precision(0.0): 0.5560885790472586 - average_precision(0.0): 0.2601114864694651 - mean_reciprocal_rank(0.0): 0.5644993523364463 - precision@1(0.0): 0.3881856540084388
Epoch 2/15
255/255 [==============================] - 10s 38ms/step - loss: 0.5718
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5460149992168801 - normalized_discounted_cumulative_gain@5(0.0): 0.6150959436466801 - mean_average_precision(0.0): 0.5714490268287737 - average_precision(0.0): 0.26451073915979445 - mean_reciprocal_rank(0.0): 0.5806340986720733 - precision@1(0.0): 0.4008438818565401
Epoch 3/15
255/255 [==============================] - 11s 43ms/step - loss: 0.4000
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.566079068275542 - normalized_discounte

In [24]:
knrm_model = mz.load_model('./krnm_pretrained_model/3')

In [25]:
%%time
knrm_model.evaluate(pred_x, pred_y)

CPU times: user 1.3 s, sys: 170 ms, total: 1.47 s
Wall time: 574 ms


{average_precision(0.0): 0.26546699091245696,
 mean_average_precision(0.0): 0.5707784967806064,
 mean_reciprocal_rank(0.0): 0.5780425739286497,
 normalized_discounted_cumulative_gain@3(0.0): 0.566079068275542,
 normalized_discounted_cumulative_gain@5(0.0): 0.6219225474436791,
 precision@1(0.0): 0.3755274261603376}